LAB 5: Wide and Deep Network Architectures

Jocelyn Rupp and Julia Rushton 

# Dataset Selection

Select a dataset similarly to lab one. That is, the dataset must be table data and must have categorical features. In terms of generalization performance, it is helpful to have a large dataset for building a wide and deep network. It is also helpful to have many different categorical features to create the embeddings and cross-product embeddings. It is fine to perform binary classification, multi-class classification, or regression. You are NOT allowed to use the census (i.e., Adult) dataset that was given as an example in class. 

# Preparation

In [1]:
import pandas as pd

In [2]:
import numpy as np

In [8]:
data_path = 'collision-data.csv'
df = pd.read_csv(data_path)

df.head()

,Unnamed: 0,longitude,latitude,SEVERITYCODE,COLLISIONTYPE,PERSONCOUNT,PEDCOUNT,PEDCYLCOUNT,VEHCOUNT,INJURIES,...,response_type,response_time,AWND,PRCP,SNOW,SNWD,TAVG,TMAX,TMIN,WSF5
0,1,-122.324322,47.613755,0,Parked Car,2,0,0,2,0,...,NaN,NaN,9.4,0.0,NaN,NaN,33.0,38,27,17.0
1,2,-122.321556,47.571558,0,Rear Ended,3,0,0,3,0,...,NaN,NaN,9.4,0.0,NaN,NaN,33.0,38,27,17.0
2,3,-122.344164,47.555769,1,Head On,2,0,0,2,1,...,NaN,NaN,9.4,0.0,NaN,NaN,33.0,38,27,17.0
3,4,-122.309400,47.608484,1,Rear Ended,2,0,0,2,1,...,Motor Vehicle Accident,36.0,9.4,0.0,NaN,NaN,33.0,38,27,17.0
4,5,-122.319746,47.674521,0,Left Turn,2,0,0,2,0,...,NaN,NaN,9.4,0.0,NaN,NaN,33.0,38,27,17.0


Define and prepare your class variables. Use proper variable representations (int, float, one-hot, etc.). Use pre-processing methods (as needed) for dimensionality reduction, scaling, etc. Remove variables that are not needed/useful for the analysis. Describe the final dataset that is used for classification/regression (include a description of any newly formed variables you created). You have the option of using tf.dataset for processing, but it is not required.

In [9]:
#
# Remove attributes that arent useful for us
df.drop(['WSF5','TMIN','TMAX', 'SNOW', 'SNWD', 'response_type', 'JUNCTIONTYPE', 'response_time'], axis = 1, inplace = True)
        
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 111882 entries, 0 to 111881
Data columns (total 26 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   Unnamed: 0            111882 non-null  int64  
 1   longitude             111882 non-null  float64
 2   latitude              111882 non-null  float64
 3   SEVERITYCODE          111882 non-null  int64  
 4   COLLISIONTYPE         111882 non-null  object 
 5   PERSONCOUNT           111882 non-null  int64  
 6   PEDCOUNT              111882 non-null  int64  
 7   PEDCYLCOUNT           111882 non-null  int64  
 8   VEHCOUNT              111882 non-null  int64  
 9   INJURIES              111882 non-null  int64  
 10  SERIOUSINJURIES       111882 non-null  int64  
 11  FATALITIES            111882 non-null  int64  
 12  INATTENTIONIND        111882 non-null  bool   
 13  UNDERINFL             111882 non-null  bool   
 14  WEATHER               111882 non-null  object 
 15  

In [11]:
# preprocessing of numeric variables

import pandas as pd
from sklearn.preprocessing import StandardScaler

# Check the columns in the dataset
print(df.columns)

# Define unnecessary columns to drop
unnecessary_columns = ['Unnamed: 0', 'SPDCASENO', 'DATE', 'TIME']
df.drop(unnecessary_columns, axis=1, errors='ignore', inplace=True)

# Standardize numeric variables
scaler = StandardScaler()
numeric_cols = ['longitude', 'latitude', 'PERSONCOUNT', 'PEDCOUNT', 'PEDCYLCOUNT', 'VEHCOUNT', 
                'INJURIES', 'SERIOUSINJURIES', 'FATALITIES', 'AWND', 'PRCP', 'TAVG']
df[numeric_cols] = scaler.fit_transform(df[numeric_cols])

# Determine variable types and describe the final dataset
variable_types = {}
for col in df.columns:
    if col in numeric_cols:
        variable_types[col] = 'Numeric'
    else:
        variable_types[col] = 'Categorical'

final_dataset_description = df.describe()

print("Variable Types:")
for col, v_type in variable_types.items():
    print(f"{col}: {v_type}")

print("\nFinal Dataset Description:")
print(final_dataset_description)

Index(['Unnamed: 0', 'longitude', 'latitude', 'SEVERITYCODE', 'COLLISIONTYPE',
       'PERSONCOUNT', 'PEDCOUNT', 'PEDCYLCOUNT', 'VEHCOUNT', 'INJURIES',
       'SERIOUSINJURIES', 'FATALITIES', 'INATTENTIONIND', 'UNDERINFL',
       'WEATHER', 'ROADCOND', 'LIGHTCOND', 'SPEEDING', 'HITPARKEDCAR',
       'SPDCASENO', 'DATE', 'TIME', 'intersection_related', 'AWND', 'PRCP',
       'TAVG'],
      dtype='object')
Variable Types:
longitude: Numeric
latitude: Numeric
SEVERITYCODE: Categorical
COLLISIONTYPE: Categorical
PERSONCOUNT: Numeric
PEDCOUNT: Numeric
PEDCYLCOUNT: Numeric
VEHCOUNT: Numeric
INJURIES: Numeric
SERIOUSINJURIES: Numeric
FATALITIES: Numeric
INATTENTIONIND: Categorical
UNDERINFL: Categorical
WEATHER: Categorical
ROADCOND: Categorical
LIGHTCOND: Categorical
SPEEDING: Categorical
HITPARKEDCAR: Categorical
intersection_related: Categorical
AWND: Numeric
PRCP: Numeric
TAVG: Numeric

Final Dataset Description:
          longitude      latitude   SEVERITYCODE   PERSONCOUNT      PEDCOUNT

Identify groups of features in your data that should be combined into cross-product features. Provide a compelling justification for why these features should be crossed (or why some features should not be crossed). 

Group 1: Severity Code and Collision Type
Group 2: Weather, Road, and Light Condition, Severity Code
Group 3: Speeding, Collision Type, Underthe influence

EXPLAIN WHY 

Choose and explain what metric(s) you will use to evaluate your algorithm’s performance. You should give a detailed argument for why this (these) metric(s) are appropriate on your data. That is, why is the metric appropriate for the task (e.g., in terms of the business case for the task). Please note: rarely is accuracy the best evaluation metric to use. Think deeply about an appropriate measure of performance.

recall because we want to mitigate false positives, would be really bad if an injury was not coded a 3 and it was

macro recall of each class

Choose the method you will use for dividing your data into training and testing (i.e., are you using Stratified 10-fold cross validation? Shuffle splits? Why?). Explain why your chosen method is appropriate or use more than one method as appropriate. Argue why your cross validation method is a realistic mirroring of how an algorithm would be used in practice. Use the method to split your data that you argue for. 

data set is sufficiently large and do not need to use cross validatoin for porving 

use a basic 80 20 split for testing

In [13]:
from sklearn.model_selection import train_test_split
import plotly


# X = df.drop('SEVERITYCODE', axis=1)  # Features
# y = df['SEVERITYCODE'].to_numpy()  # Target variable

## ask about plotting recall with multiclass
#y = (y > 1).astype(int) 

# Split the data into training and testing sets
df_train, df_test = train_test_split(df, test_size=0.2, random_state=42)

# Modeling

In [19]:
#!pip install tensorflow

  Obtaining dependency information for tensorflow from https://files.pythonhosted.org/packages/b8/75/ce4d8eeb1fb100726634358411bc4a8b12f889f6ce560b0973c0a5dbac39/tensorflow-2.16.1-cp311-cp311-macosx_10_15_x86_64.whl.metadata
  Obtaining dependency information for absl-py>=1.0.0 from https://files.pythonhosted.org/packages/a2/ad/e0d3c824784ff121c03cc031f944bc7e139a8f1870ffd2845cc2dd76f6c4/absl_py-2.1.0-py3-none-any.whl.metadata
  Obtaining dependency information for astunparse>=1.6.0 from https://files.pythonhosted.org/packages/2b/03/13dde6512ad7b4557eb792fbcf0c653af6076b81e5941d36ec61f7ce6028/astunparse-1.6.3-py2.py3-none-any.whl.metadata
  Obtaining dependency information for flatbuffers>=23.5.26 from https://files.pythonhosted.org/packages/41/f0/7e988a019bc54b2dbd0ad4182ef2d53488bb02e58694cd79d61369e85900/flatbuffers-24.3.25-py2.py3-none-any.whl.metadata
  Obtaining dependency information for gast!=0.5.0,!=0.5.1,!=0.5.2,>=0.2.1 from https://files.pythonhosted.org/packages/fa/39/5aae5

In [20]:
import tensorflow as tf

batch_size = 64

def create_dataset_from_dataframe(df_input, outcome):

    df = df_input.copy()
    labels = df[outcome]

    df = {key: value.values[:,np.newaxis] for key, value in df_input[[col for col in df.columns if col != outcome]].items()}

    # create the Dataset here
    ds = tf.data.Dataset.from_tensor_slices((dict(df), labels))
    
    # now enable batching and prefetching
    ds = ds.batch(batch_size)
    ds = ds.prefetch(batch_size)
    
    return ds

ds_train = create_dataset_from_dataframe(df_train, "SEVERITYCODE")
ds_test = create_dataset_from_dataframe(df_test, "SEVERITYCODE")

2024-04-09 18:29:37.546035: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


 Create at least three combined wide and deep networks to classify your data using Keras (this total of "three" includes the model you will train in the next step of the rubric). Visualize the performance of the network on the training data and validation data in the same plot versus the training iterations. 
Note: you can use the "history" return parameter that is part of Keras "fit" function to easily access this data.

In [ ]:
##make boolean yes or no

In [22]:
from tensorflow.keras.utils import FeatureSpace

# Example One: Just lump everything together, and concatenate
feature_space = FeatureSpace(
    features={
        # Categorical feature encoded as string
        ##CHANGE
        "SEVERITYCODE": FeatureSpace.string_categorical(num_oov_indices=0),
        "COLLISIONTYPE": FeatureSpace.string_categorical(num_oov_indices=0),
        "INATTENTIONIND": FeatureSpace.string_categorical(num_oov_indices=0),
        "UNDERINFL": FeatureSpace.string_categorical(num_oov_indices=0),
        "WEATHER": FeatureSpace.string_categorical(num_oov_indices=0),
        "ROADCOND": FeatureSpace.string_categorical(num_oov_indices=0),
        "LIGHTCOND": FeatureSpace.string_categorical(num_oov_indices=0),
        "HITPARKEDCAR": FeatureSpace.string_categorical(num_oov_indices=0),
        "SPEEDING": FeatureSpace.string_categorical(num_oov_indices=0),
        "intersection_related": FeatureSpace.string_categorical(num_oov_indices=0),
        

    },
    output_mode="concat", # can also be a dict, processed internally
)


In [23]:
# create a version of the dataset that can be iterated without labels
train_ds_with_no_labels = ds_train.map(lambda x, _: x)  
feature_space.adapt(train_ds_with_no_labels)
test_ds_with_no_labels = ds_test.map(lambda x, _: x)  
feature_space.adapt(test_ds_with_no_labels)

KeyError: 'SEVERITYCODE'

Investigate generalization performance by altering the number of layers in the deep branch of the network. Try at least two models (this "two" includes the wide and deep model trained from the previous step). Use the method of cross validation and evaluation metric that you argued for at the beginning of the lab to answer: What model with what number of layers performs superiorly? Use proper statistical methods to compare the performance of different models.

Compare the performance of your best wide and deep network to a standard multi-layer perceptron (MLP). Alternatively, you can compare to a network without the wide branch (i.e., just the deep network). For classification tasks, compare using the receiver operating characteristic and area under the curve. For regression tasks, use Bland-Altman plots and residual variance calculations.  Use proper statistical methods to compare the performance of different models.  

# Exceptional Work

5000 students: You have free reign to provide additional analyses.
One idea (required for 7000 level students): Capture the embedding weights from the deep network and (if needed) perform dimensionality reduction on the output of these embedding layers (only if needed). That is, pass the observations into the network, save the embedded weights (called embeddings), and then perform  dimensionality reduction in order to visualize results. Visualize and explain any clusters in the data.
